# feature engineering of recognized values (Label == 1)

In [9]:
from pyspark.sql.functions import *
d = spark.read.csv('/user/maxnet/database/sig.db/data_ct_res_hostname/*',sep='\x01')
d = d.withColumnRenamed('_c1','val').withColumnRenamed('_c3','osid')
d = d.select('val','osid').dropna().distinct()
#d1 = d.filter(d.osid == 0).distinct().withColumnRenamed('osid','label').select('val',col('label').cast('float'))
d2 = d.filter(d.osid != 0).distinct().withColumn('label',lit(1)).select('val',col('label').cast('float')).distinct()

#d2 = d2.sample(0.7)

d2 = d2.withColumn('f1',length(col('val')))

d2 = d2.withColumn('f2',when(d2.val.startswith('A')|d2.val.startswith('B')|d2.val.startswith('C')\
                             |d2.val.startswith('D')|d2.val.startswith('E')|d2.val.startswith('F')\
                             |d2.val.startswith('G')|d2.val.startswith('H')|d2.val.startswith('I')\
                             |d2.val.startswith('J')|d2.val.startswith('K')|d2.val.startswith('L')\
                             |d2.val.startswith('M')|d2.val.startswith('N')|d2.val.startswith('O')\
                             |d2.val.startswith('P')|d2.val.startswith('Q')|d2.val.startswith('R')\
                             |d2.val.startswith('S')|d2.val.startswith('T')|d2.val.startswith('U')\
                             |d2.val.startswith('V')|d2.val.startswith('W')|d2.val.startswith('X')\
                             |d2.val.startswith('Y')|d2.val.startswith('Z'),1).otherwise(0))

import re

num_regex = re.compile(r'[0-9]') #数字
xiaoxiezimu_regex = re.compile(r'[a-z]')#小写字母
daxiezimu_regex = re.compile(r'[A-Z]') #大写字母 
#hanzi_regex = re.compile(r'[\u4E00-\u9FA5]')#汉字

from pyspark.sql.functions import udf
num = udf(lambda x: len(num_regex.findall(x)))
xiaoxie = udf(lambda x: len(xiaoxiezimu_regex.findall(x)))
daxie = udf(lambda x: len(daxiezimu_regex.findall(x)))

d2 = d2.withColumn('f3',num('val'))
d2 = d2.withColumn('f4',xiaoxie('val'))
d2 = d2.withColumn('f5',daxie('val'))

# 特征字符串长度 f1
# 首字母是否大写 f2
# 数字字符数量   f3
# 小写字母数量   f4
# 大写字母数量   f5
# 特殊符号-_:的数量 f6
# 空格的数量 f7


# 统计下划线个数
def xiahuaxian_count(s):
    xiahuaxian_counts=0
    for c in s:
        xiahuaxian_split_list = c.split('_')
        xiahuaxian_counts += len(xiahuaxian_split_list) - 1
    return xiahuaxian_counts

# 统计中划线个数
def zhonghuaxian_count(s):
    zhonghuaxian_counts=0
    for c in s:
        zhonghuaxian_split_list = c.split('-')
        zhonghuaxian_counts += len(zhonghuaxian_split_list) - 1
    return zhonghuaxian_counts

# 统计冒号个数
def maohao_count(s):
    maohao_counts=0
    for c in s:
        maohao_split_list = c.split(':')
        maohao_counts += len(maohao_split_list) - 1
    return maohao_counts

def teshu_count(s):
    teshu_counts=0
    a_counts=0
    b_counts=0
    c_counts=0
    for c in s:
        a_split_list = c.split('_')
        a_counts += len(a_split_list) - 1
        
        b_split_list = c.split('-')
        b_counts += len(b_split_list) - 1
        
        c_split_list = c.split(':')
        c_counts += len(c_split_list) - 1
        
        teshu_counts = a_counts + b_counts + c_counts
    return teshu_counts
        

# 统计空格个数
def space_count(s):
    space_counts=0
    for c in s:
        space_split_list = c.split(' ')
        space_counts += len(space_split_list) - 1
    return space_counts

teshu = udf(lambda x: teshu_count(x))
kongge = udf(lambda x: space_count(x))


d2 = d2.withColumn('f6',teshu('val'))
d2 = d2.withColumn('f7',kongge('val'))

d2 = d2.select('val','label',col('f1').cast('float'),col('f2').cast('float'),col('f3').cast('float'),col('f4').cast('float'),col('f5').cast('float'),col('f6').cast('float'),col('f7').cast('float'))

#dd = d2.repartition(1)

#dd.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/recognized/',compression='gzip')

# feature engineering of unrecognized values (Label == 0)

In [2]:
from pyspark.sql.functions import *
d = spark.read.csv('/user/maxnet/database/sig.db/data_ct_res_hostname/*',sep='\x01')
d = d.withColumnRenamed('_c1','val').withColumnRenamed('_c3','osid')
d = d.select('val','osid').dropna().distinct()
d1 = d.filter(d.osid == 0).distinct().withColumnRenamed('osid','label').select('val',col('label').cast('float'))
#d2 = d.filter(d.osid != 0).distinct().withColumn('label',lit(1)).select('val',col('label').cast('float')).distinct()

#d2 = d2.sample(0.7)

d1 = d1.withColumn('f1',length(col('val')))

d1 = d1.withColumn('f2',when(d1.val.startswith('A')|d1.val.startswith('B')|d1.val.startswith('C')\
                             |d1.val.startswith('D')|d1.val.startswith('E')|d1.val.startswith('F')\
                             |d1.val.startswith('G')|d1.val.startswith('H')|d1.val.startswith('I')\
                             |d1.val.startswith('J')|d1.val.startswith('K')|d1.val.startswith('L')\
                             |d1.val.startswith('M')|d1.val.startswith('N')|d1.val.startswith('O')\
                             |d1.val.startswith('P')|d1.val.startswith('Q')|d1.val.startswith('R')\
                             |d1.val.startswith('S')|d1.val.startswith('T')|d1.val.startswith('U')\
                             |d1.val.startswith('V')|d1.val.startswith('W')|d1.val.startswith('X')\
                             |d1.val.startswith('Y')|d1.val.startswith('Z'),1).otherwise(0))

import re

num_regex = re.compile(r'[0-9]') #数字
xiaoxiezimu_regex = re.compile(r'[a-z]')#小写字母
daxiezimu_regex = re.compile(r'[A-Z]') #大写字母 
#hanzi_regex = re.compile(r'[\u4E00-\u9FA5]')#汉字

from pyspark.sql.functions import udf
num = udf(lambda x: len(num_regex.findall(x)))
xiaoxie = udf(lambda x: len(xiaoxiezimu_regex.findall(x)))
daxie = udf(lambda x: len(daxiezimu_regex.findall(x)))

d1 = d1.withColumn('f3',num('val'))
d1 = d1.withColumn('f4',xiaoxie('val'))
d1 = d1.withColumn('f5',daxie('val'))

# 特征字符串长度 f1
# 首字母是否大写 f2
# 数字字符数量   f3
# 小写字母数量   f4
# 大写字母数量   f5
# 特殊符号-_:的数量 f6
# 空格的数量 f7


# 统计下划线个数
def xiahuaxian_count(s):
    xiahuaxian_counts=0
    for c in s:
        xiahuaxian_split_list = c.split('_')
        xiahuaxian_counts += len(xiahuaxian_split_list) - 1
    return xiahuaxian_counts

# 统计中划线个数
def zhonghuaxian_count(s):
    zhonghuaxian_counts=0
    for c in s:
        zhonghuaxian_split_list = c.split('-')
        zhonghuaxian_counts += len(zhonghuaxian_split_list) - 1
    return zhonghuaxian_counts

# 统计冒号个数
def maohao_count(s):
    maohao_counts=0
    for c in s:
        maohao_split_list = c.split(':')
        maohao_counts += len(maohao_split_list) - 1
    return maohao_counts

def teshu_count(s):
    teshu_counts=0
    a_counts=0
    b_counts=0
    c_counts=0
    for c in s:
        a_split_list = c.split('_')
        a_counts += len(a_split_list) - 1
        
        b_split_list = c.split('-')
        b_counts += len(b_split_list) - 1
        
        c_split_list = c.split(':')
        c_counts += len(c_split_list) - 1
        
        teshu_counts = a_counts + b_counts + c_counts
    return teshu_counts
        

# 统计空格个数
def space_count(s):
    space_counts=0
    for c in s:
        space_split_list = c.split(' ')
        space_counts += len(space_split_list) - 1
    return space_counts

teshu = udf(lambda x: teshu_count(x))
kongge = udf(lambda x: space_count(x))


d1 = d1.withColumn('f6',teshu('val'))
d1 = d1.withColumn('f7',kongge('val'))


d1 = d1.select('val','label',col('f1').cast('float'),col('f2').cast('float'),col('f3').cast('float'),col('f4').cast('float'),col('f5').cast('float'),col('f6').cast('float'),col('f7').cast('float'))

#ddd = d1.repartition(1)

#ddd.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/unrecognized/',compression='gzip')

# Feature Importance Analysis

In [1]:
from pyspark.sql.functions import *

d1 = spark.read.parquet('/data/user/hive/warehouse/ian/feature/unrecognized/*')
d2 = spark.read.parquet('/data/user/hive/warehouse/ian/feature/recognized/*')

#d1 = d1.withColumn('id1',monotonically_increasing_id())
#d2 = d2.withColumn('id2',monotonically_increasing_id())

In [13]:
d1.count(),d2.count()

(13366199, 118581095)

In [14]:
13366199+118581095

131947294

In [15]:
d1.columns

['val', 'label', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7']

In [16]:
d2.columns

['val', 'label', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7']

In [17]:
dd = d1.union(d2)

In [18]:
dd.count()

131947294

In [19]:
dd.corr('label','f1') ## 特征字符串长度

0.4034425936156893

In [20]:
dd.corr('label','f2') # 首字母是否大写

-0.07627069092079923

In [21]:
dd.corr('label','f3') # 数字字符数量

0.07186411864088836

In [22]:
dd.corr('label','f4') ## 小写字母数量

0.3887611231602581

In [23]:
dd.corr('label','f5') # 大写字母数量

-0.19934939410810917

In [24]:
dd.corr('label','f6') ## 特殊符号-_:的数量

0.3214068297187456

In [25]:
dd.corr('label','f7') # 空格的数量

-0.01872019384940059

In [26]:
d1.describe('f1').show()

+-------+------------------+
|summary|                f1|
+-------+------------------+
|  count|          13366199|
|   mean|16.745152081006726|
| stddev|  8.51290374526239|
|    min|               1.0|
|    max|              64.0|
+-------+------------------+



In [27]:
d2.describe('f1').show()

+-------+-----------------+
|summary|               f1|
+-------+-----------------+
|  count|        118581095|
|   mean| 22.8576718067918|
| stddev|3.361524311119653|
|    min|              3.0|
|    max|             64.0|
+-------+-----------------+



In [28]:
d1.describe('f4').show()

+-------+-----------------+
|summary|               f4|
+-------+-----------------+
|  count|         13366199|
|   mean|3.770518604428978|
| stddev|5.273092558697898|
|    min|              0.0|
|    max|             64.0|
+-------+-----------------+



In [29]:
d2.describe('f4').show()

+-------+------------------+
|summary|                f4|
+-------+------------------+
|  count|         118581095|
|   mean|10.352725339566142|
| stddev| 4.638554647787675|
|    min|               0.0|
|    max|              62.0|
+-------+------------------+



In [30]:
d1.describe('f6').show()

+-------+------------------+
|summary|                f6|
+-------+------------------+
|  count|          13366199|
|   mean|0.4301927571181605|
| stddev|0.8938209331999687|
|    min|               0.0|
|    max|              58.0|
+-------+------------------+



In [31]:
d2.describe('f6').show()

+-------+------------------+
|summary|                f6|
+-------+------------------+
|  count|         118581095|
|   mean|1.2792700893848215|
| stddev|0.7374773238050477|
|    min|               0.0|
|    max|              39.0|
+-------+------------------+



# feature_count

In [37]:
from pyspark.sql.functions import *
d = spark.read.csv('/user/maxnet/database/sig.db/data_ct_res_hostname/*',sep='\x01')

d = d.withColumnRenamed('_c1','val').withColumnRenamed('_c3','osid')
d = d.select('val','osid')

d1 = d.filter(d.osid == 0).withColumnRenamed('osid','label').select('val',col('label').cast('float')).dropna()
d2 = d.filter(d.osid != 0).withColumn('label',lit(1)).select('val',col('label').cast('float')).dropna()

df = d1.union(d2)

g = df.groupBy('val').count().withColumnRenamed('val','val1')

dd = df.join(g,df.val == g.val1,how='left_outer')

dd = dd.withColumnRenamed('count','val_count')

dd.corr('label','val_count')

-0.7132714610776969

In [79]:
dd.filter(dd.label == 1).select('val_count').distinct().describe().show(truncate=False)

+-------+------------------+
|summary|val_count         |
+-------+------------------+
|count  |3525              |
|mean   |40473.719432624115|
|stddev |291219.4611998903 |
|min    |1                 |
|max    |14196282          |
+-------+------------------+



In [80]:
dd.filter(dd.label == 0).select('val_count').distinct().describe().show(truncate=False)

+-------+-----------------+
|summary|val_count        |
+-------+-----------------+
|count  |1519             |
|mean   |33510.17577353522|
|stddev |831804.8084261566|
|min    |1                |
|max    |29398040         |
+-------+-----------------+



# feature importance

In [44]:
from pyspark.sql.functions import *

d1 = spark.read.parquet('/data/user/hive/warehouse/ian/feature/unrecognized/*')
d2 = spark.read.parquet('/data/user/hive/warehouse/ian/feature/recognized/*')

In [45]:
df = d1.union(d2)

In [35]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
unlist = udf(lambda x: float(list(x)[0]), DoubleType())

In [36]:
from pyspark.ml.feature import MinMaxScaler,StandardScaler,VectorAssembler

In [39]:
vec_f1 = VectorAssembler(inputCols=['f4'],outputCol='vec_f1')
f1_v = vec_f1.transform(df)
minmax_f1 = MinMaxScaler(inputCol='vec_f1',outputCol='minmax_f1')
minmax = minmax_f1.fit(f1_v)
mm_f1 = minmax.transform(f1_v).select('minmax_f1').withColumn('mm_f1',unlist('minmax_f1')).select('mm_f1')
mm_f1.select(stddev('mm_f1')).show()

+-------------------+
| stddev_samp(mm_f1)|
+-------------------+
|0.07982150640018788|
+-------------------+



In [40]:
vec_f1 = VectorAssembler(inputCols=['f5'],outputCol='vec_f1')
f1_v = vec_f1.transform(df)
minmax_f1 = MinMaxScaler(inputCol='vec_f1',outputCol='minmax_f1')
minmax = minmax_f1.fit(f1_v)
mm_f1 = minmax.transform(f1_v).select('minmax_f1').withColumn('mm_f1',unlist('minmax_f1')).select('mm_f1')
mm_f1.select(stddev('mm_f1')).show()

+-------------------+
| stddev_samp(mm_f1)|
+-------------------+
|0.05313065963090275|
+-------------------+



In [41]:
vec_f1 = VectorAssembler(inputCols=['f6'],outputCol='vec_f1')
f1_v = vec_f1.transform(df)
minmax_f1 = MinMaxScaler(inputCol='vec_f1',outputCol='minmax_f1')
minmax = minmax_f1.fit(f1_v)
mm_f1 = minmax.transform(f1_v).select('minmax_f1').withColumn('mm_f1',unlist('minmax_f1')).select('mm_f1')
mm_f1.select(stddev('mm_f1')).show()

+-------------------+
| stddev_samp(mm_f1)|
+-------------------+
|0.01374280408348266|
+-------------------+



In [42]:
vec_f1 = VectorAssembler(inputCols=['f7'],outputCol='vec_f1')
f1_v = vec_f1.transform(df)
minmax_f1 = MinMaxScaler(inputCol='vec_f1',outputCol='minmax_f1')
minmax = minmax_f1.fit(f1_v)
mm_f1 = minmax.transform(f1_v).select('minmax_f1').withColumn('mm_f1',unlist('minmax_f1')).select('mm_f1')
mm_f1.select(stddev('mm_f1')).show()

+--------------------+
|  stddev_samp(mm_f1)|
+--------------------+
|0.001091062901651...|
+--------------------+

